In [1]:
# import numpy as np
# from lines.Line import Line
# from prototypes.models.PrototypeClassifierModels import CLASSIFIER_MODEL_2NN
# from training.CrossEntropyTraining import ClassifierTrainerWithCrossEntropyNoGen
# from datautils.LEOPARDEncoderUtils import get_labelled_LEOPARD_training_data as get_labelled_training_data
# from lines.LineGenerator import LineGenerator
# # from evaluation.Evaluator import Evaluator

# test_params = {
#   'type': 'cross_entropy',
#   'encoder': "bert",
# }

# trainer_params = {
#   'shuffle': True,
#   'num_workers': 0
# }

# batch_size = {
#   4: 4, 8: 8, 16: 12
# }

# epochs = {
#   4: {
#     2: 15, 3: 30, 4: 45, 5: 60, 6: 75
#   },
#   8: {
#     2: 30, 3: 40, 4: 50, 5: 70, 6: 100
#   },
#   16: {
#     2: 15, 3: 25, 4: 35, 5: 45, 6: 55
#   },
# }

# learning_rates = {
#   4: 5e-3,
#   8: 1e-2,
#   16: 5e-2
# }

# training_params = {
#   'encoder': "bert",
#   'epochs': epochs,
#   'warmupSteps': 100,
#   'reduction': 'none',
#   'learning_rate': learning_rates,
#   'printValidationPlot': True,
#   'printValidationLoss': False
# }

# for category in ["rating_dvd"]:
#     training_params['category'] = category
#     for episode in range(10):
#         training_params['episode'] = episode
#         for shot in [4]:
#             training_params['shot'] = shot
#             trainer_params['batch_size'] = batch_size[shot]
#             training_encodings, training_labels, label_keys = get_labelled_training_data(category, shot, episode)
            
#             if training_encodings.shape[0] == 0:
#                 continue
            
#             training_set = {}
#             training_set['encodings'] = training_encodings
#             training_set['labels'] = training_labels
            
#             lineGenerator = LineGenerator(training_set, CLASSIFIER_MODEL_2NN)
#             classifierTrainer = ClassifierTrainerWithCrossEntropyNoGen(lineGenerator, trainer_params, training_params, label_keys)
#             classifierTrainer.trainPrototypes(training_params, trainer_params, training_set)
#             lines = classifierTrainer.getLines()
            
# #             evaluator = Evaluator(label_keys, test_params)
# #             evaluator.evaluate(category, shot, episode)

In [2]:
from datasets import load_dataset
from training_datasets.GLUEDataset import GLUEDataset
import learn2learn as l2l

cola = load_dataset('glue','cola')
print(cola)
sst2 = load_dataset('glue','sst2')
print(sst2)
pt_cola = GLUEDataset([cola, sst2], 'train')

dataset_meta = l2l.data.MetaDataset(pt_cola)


/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1357.38it/s]


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})


Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1293.21it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})


In [3]:
from training_datasets.GLUEMetaDataset import GLUEMetaDataset

gm_ds = GLUEMetaDataset(pt_cola, k=2, numTasks=1)
task = gm_ds.getTask()
print(task)

[('[CLS] Fanny continued talking although in came Aunt Norris. [SEP]', "[CLS] Which poem did you hear Homer's recital of last night? [SEP]", '[CLS] He figured out the answer. [SEP]', '[CLS] You may pick any five flowers. [SEP]', '[CLS] is that it jams too many prefabricated story elements into the running time  [SEP]', "[CLS] rubbo 's dumbed-down tactics  [SEP]", '[CLS] inspiring , ironic ,  [SEP]', '[CLS] childhood imagination  [SEP]'), tensor([0, 0, 1, 1, 2, 2, 3, 3])]


In [4]:
# episodes = []
# for i in range(1):
#     episodes.append(gm_ds.getTask())
# for i in range(len(episodes)):
#     classes = len(set(episodes[i][1].tolist()))
#     # print(classes)
#     print(episodes[i][0], episodes[i][1], "\n")

In [5]:
from training.models.FOMAML import FOMAML

params = {"batch_size":1}
fomaml = FOMAML(params=params, labelKeys=None)
fomaml.trainPrototypes(gm_ds, params)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predicti

tensor([0, 0, 1, 1, 2, 2, 3, 3])
computing lines now
Lines have been computed
tensor([[ 0.0134,  0.0291,  0.0236,  ...,  0.0284, -0.0168,  0.0337],
        [ 0.0079,  0.0199,  0.0278,  ..., -0.0227, -0.0009,  0.0160],
        [-0.0046,  0.0231,  0.0022,  ..., -0.0337,  0.0307,  0.0066],
        ...,
        [-0.0234,  0.0229, -0.0105,  ...,  0.0147, -0.0197,  0.0257],
        [-0.0060, -0.0358,  0.0249,  ...,  0.0108,  0.0359,  0.0251],
        [-0.0148,  0.0262,  0.0279,  ...,  0.0252,  0.0167, -0.0170]])
tensor([-0.0185,  0.0137, -0.0053, -0.0127,  0.0317,  0.0320,  0.0306,  0.0197,
        -0.0139,  0.0136,  0.0053, -0.0043, -0.0205,  0.0096, -0.0223, -0.0088,
         0.0027, -0.0156,  0.0338,  0.0105,  0.0173,  0.0322,  0.0334,  0.0284,
        -0.0263, -0.0243,  0.0183, -0.0357,  0.0246, -0.0202,  0.0070,  0.0146,
        -0.0102,  0.0142,  0.0239,  0.0068, -0.0204,  0.0043,  0.0184,  0.0128,
         0.0085, -0.0160, -0.0185, -0.0134, -0.0084,  0.0049, -0.0327,  0.0332,
        

In [6]:
# import learn2learn as l2l
# from learn2learn.algorithms import LightningMAML
# from learn2learn.utils.lightning import EpisodicBatcher
# import pytorch_lightning as pl

# dict_args = {}
# dict_args['first_order'] = True

# tasksets = l2l.vision.benchmarks.get_tasksets('omniglot')
# model = l2l.vision.models.OmniglotFC(28**2, 10)
# maml = LightningMAML(model, adaptation_lr=0.1, **dict_args)
# episodic_data = EpisodicBatcher(tasksets.train, tasksets.validation, tasksets.test)
# trainer = pl.Trainer()
# trainer.fit(maml, episodic_data)